Training notebook for [Diff-SVC](https://github.com/prophesier/diff-svc) originally made by [justinjohn-03](https://github.com/justinjohn0306)
 Modified by [奕晨](https://twitter.com/nekrothecorpse) of [Archivoice](https://github.com/archivoice) and currently maintained by [haru0l](https://twitter.com/mscoocoo2)

This is the basic version of the notebook. This notebook assumes that you have READ the documentation and no support will be given.

QOL improvements has been removed in this version due to a certain someone that won't credit the usage of the program.

# Preparation

In [ ]:
import os
from IPython.display import clear_output

!rm -rf sample_data
!python -m pip install --upgrade pip wheel --quiet
!pip uninstall gdown -y --quiet
!pip install --pre torchtext==0.6.0 --no-deps --quiet

#@markdown ###Model sample rate
#@markdown Please choose if you want to train a 24kHz model or a 44.1kHz model.
sample_rate = '44.1kHz' #@param ["24kHz", "44.1kHz"]

print('Installing Diff-SVC')
!git clone -q https://github.com/prophesier/diff-svc

%cd "diff-svc"
!pip install -r requirements_short.txt --quiet
!pip install tensorboard<2.9,>=2.8 --quiet
#!pip install --upgrade numpy==1.23.0 scipy==1.9.3 --quiet
%reload_ext tensorboard

%mkdir -p checkpoints

hifigan_24k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/models_24khz/hifigan_24k.zip"
hifigan_44k = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/hifigan_44k.zip"
checkpoints = "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/start/checkpoints.zip"

!wget {checkpoints}
!unzip checkpoints.zip
!rm checkpoints.zip

if sample_rate == "44.1kHz":
	!wget {hifigan_44k} -q
	!unzip hifigan_44k.zip -d checkpoints
	!rm hifigan_44k.zip
	config_path = "training/config_nsf.yaml"
else:
	!wget {hifigan_24k}
	!unzip hifigan_24k.zip
	!rm hifigan_24k.zip
	config_path = "training/config.yaml"
clear_output()
print('Done!')

You are now expected to extract the wavs into data/raw/(singer_name_here)

# Training

In [ ]:
#@title #Pre-processing
#@markdown This just converts your data into mel spectograms

os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python preprocessing/binarize.py --config {config_path}

In [ ]:
#@title #Training
#@markdown Trains your model.

os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python run.py --config {config_path} --exp_name $singer_name --reset